In [1]:
import time
import yaml
from tqdm.notebook import tqdm
from collections import defaultdict

import pandas as pd
import OpenDartReader
import FinanceDataReader as fdr

import psycopg
from psycopg import sql
from pathlib import Path
from tqdm.notebook import tqdm

필요한 계정 항목

|sj_div|sj_nm|account_nm
|---|---|---
|BS|재무상태표|유동자산
|BS|재무상태표|현금및현금성자산
|BS|재무상태표|매출채권
|BS|재무상태표|선급비용
|BS|재무상태표|재고자산
|BS|재무상태표|비유동자산
|BS|재무상태표|유형자산
|BS|재무상태표|무형자산
|BS|재무상태표|자산총계
|BS|재무상태표|유동부채
|BS|재무상태표|매입채무
|BS|재무상태표|단기차입금
|BS|재무상태표|선수금
|BS|재무상태표|비유동부채
|BS|재무상태표|사채
|BS|재무상태표|장기차입금
|BS|재무상태표|부채총계
|BS|재무상태표|자본총계
|BS|재무상태표|부채와자본총계
|IS|손익계산서|수익(매출액)
|IS|손익계산서|매출원가
|IS|손익계산서|매출총이익
|IS|손익계산서|판매비와관리비
|IS|손익계산서|영업이익
|IS|손익계산서|금융수익
|IS|손익계산서|금융비용
|IS|손익계산서|법인세비용차감전순이익(손실)
|IS|손익계산서|법인세비용
|IS|손익계산서|계속영업이익(손실)
|IS|손익계산서|당기순이익(손실)
|CF|현금흐름표|영업활동 현금흐름
|CF|현금흐름표|영업에서 창출된 현금흐름
|CF|현금흐름표|투자활동 현금흐름
|CF|현금흐름표|재무활동 현금흐름
|CF|현금흐름표|외화환산으로 인한 현금의 변동
|CF|현금흐름표|현금및현금성자산의 순증감
|CF|현금흐름표|기초의 현금및현금성자산
|CF|현금흐름표|기말의 현금및현금성자산

In [2]:
# Load settings
src_path = Path('.').absolute().parent
with (src_path / 'setting_files' / 'settings.yml').open('r') as file:
    settings = yaml.full_load(file)

dart_settings = settings['opendart']
dart = OpenDartReader(dart_settings['apikey'])

|재무제표구분|재무제표명칭|개별/연결|표시방법|세전세후
|---|---|---|---|---|
|BS1|재무상태표|연결|유동/비유동법|
|BS2|재무상태표|개별|유동/비유동법|
|BS3|재무상태표|연결|유동성배열법|
|BS4|재무상태표|개별|유동성배열법|
|IS1|별개의 손익계산서|연결|기능별분류|
|IS2|별개의 손익계산서|개별|기능별분류|
|IS3|별개의 손익계산서|연결|성격별분류|
|IS4|별개의 손익계산서|개별|성격별분류|
|CIS1|포괄손익계산서|연결|세후|
|CIS2|포괄손익계산서|개별|세후|
|CIS3|포괄손익계산서|연결|세전|
|CIS4|포괄손익계산서|개별|세전|
|DCIS1|단일 포괄손익계산서|연결|기능별분류|세후포괄손익
|DCIS2|단일 포괄손익계산서|개별|기능별분류|세후포괄손익
|DCIS3|단일 포괄손익계산서|연결|기능별분류|세전
|DCIS4|단일 포괄손익계산서|개별|기능별분류|세전
|DCIS5|단일 포괄손익계산서|연결|성격별분류|세후포괄손익
|DCIS6|단일 포괄손익계산서|개별|성격별분류|세후포괄손익
|DCIS7|단일 포괄손익계산서|연결|성격별분류|세전
|DCIS8|단일 포괄손익계산서|개별|성격별분류|세전
|CF1|현금흐름표|연결|직접법|
|CF2|현금흐름표|개별|직접법|
|CF3|현금흐름표|연결|간접법|
|CF4|현금흐름표|개별|간접법|
|SCE1|자본변동표|연결||
|SCE2|자본변동표|개별||

In [3]:
xbrl_bs = dart.xbrl_taxonomy('BS1')
xbrl_is = dart.xbrl_taxonomy('IS1')
xbrl_cis = dart.xbrl_taxonomy('CIS1')
xbrl_cf = dart.xbrl_taxonomy('CF1')

## Tutorial for CSVtoRDF

https://stackoverflow.com/questions/43524943/creating-rdf-file-using-csv-file-as-input

In [4]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD #most common namespaces
import urllib.parse #for parsing strings to URI's
import pandas as pd

In [5]:
url='https://raw.githubusercontent.com/KRontheWeb/csv2rdf-tutorial/master/example.csv'
df=pd.read_csv(url,sep=";",quotechar='"')

In [6]:
df

,Name,Address,Place,Country,Age,Hobby,Favourite Colour
0,John,Dam 52,Amsterdam,The Netherlands,32,Fishing,Blue
1,Jenny,Leidseplein 2,Amsterdam,The Netherlands,12,Dancing,Mauve
2,Jill,52W Street 5,Amsterdam,United States of America,28,Carpentry,Cyan
3,Jake,12E Street 98,Amsterdam,United States of America,42,Ballet,Purple


In [7]:
g = Graph()
ppl = Namespace('http://example.com/people/')
loc = Namespace('http://mylocations.org/addresses/')
schema = Namespace('http://schema.org/')

In [8]:
for index, row in df.iterrows():
    g.add((URIRef(ppl+row['Name']), RDF.type, FOAF.Person))
    g.add((URIRef(ppl+row['Name']), URIRef(schema+'name'), Literal(row['Name'], datatype=XSD.string) ))
    g.add((URIRef(ppl+row['Name']), FOAF.age, Literal(row['Age'], datatype=XSD.integer) ))
    g.add((URIRef(ppl+row['Name']), URIRef(schema+'address'), Literal(row['Address'], datatype=XSD.string) ))
    g.add((URIRef(loc+urllib.parse.quote(row['Address'])), URIRef(schema+'name'), Literal(row['Address'], datatype=XSD.string) ))

In [10]:
print(g.serialize(format='turtle'))
g.serialize('example.ttl',format='turtle')

@prefix ns1: <http://xmlns.com/foaf/0.1/> .
@prefix ns2: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.com/people/Jake> a ns1:Person ;
    ns2:address "12E Street 98"^^xsd:string ;
    ns2:name "Jake"^^xsd:string ;
    ns1:age 42 .

<http://example.com/people/Jenny> a ns1:Person ;
    ns2:address "Leidseplein 2"^^xsd:string ;
    ns2:name "Jenny"^^xsd:string ;
    ns1:age 12 .

<http://example.com/people/Jill> a ns1:Person ;
    ns2:address "52W Street 5"^^xsd:string ;
    ns2:name "Jill"^^xsd:string ;
    ns1:age 28 .

<http://example.com/people/John> a ns1:Person ;
    ns2:address "Dam 52"^^xsd:string ;
    ns2:name "John"^^xsd:string ;
    ns1:age 32 .

<http://mylocations.org/addresses/12E%20Street%2098> ns2:name "12E Street 98"^^xsd:string .

<http://mylocations.org/addresses/52W%20Street%205> ns2:name "52W Street 5"^^xsd:string .

<http://mylocations.org/addresses/Dam%2052> ns2:name "Dam 52"^^xsd:string .

<http://mylocations.org/addre

<Graph identifier=N42150f30038f4cdaa20c991a4360946a (<class 'rdflib.graph.Graph'>)>